In [68]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

In [69]:
movies = pd.read_csv("D:\GITHUB\Rec_sys\Dtaset\ml-latest-small\movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [70]:
ratings =pd.read_csv("D:/GITHUB/Rec_sys/Dtaset/ml-latest-small/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [71]:
mean_rating_user = ratings.groupby('userId').agg({'rating':['mean']}).reset_index()
mean_rating_film = ratings.groupby('movieId').agg({'rating':['mean']}).reset_index()
mean_rating_film = mean_rating_film.rename(columns = {'rating': 'rating_f'})

In [72]:
joined_ratings = ratings.join(mean_rating_user.set_index('userId'), on='userId')


C:\Users\lokky\AppData\Local\Temp\ipykernel_23084\2357977885.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  joined_ratings = ratings.join(mean_rating_user.set_index('userId'), on='userId')


In [73]:
joined_ratings = joined_ratings.join(mean_rating_film.set_index('movieId'), on='movieId')

C:\Users\lokky\AppData\Local\Temp\ipykernel_23084\965712231.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  joined_ratings = joined_ratings.join(mean_rating_film.set_index('movieId'), on='movieId')


In [74]:
joined_ratings.head()

,userId,movieId,rating,timestamp,"(rating, mean)","(rating_f, mean)"
0,1,1,4.0,964982703,4.366379,3.920930
1,1,3,4.0,964981247,4.366379,3.259615
2,1,6,4.0,964982224,4.366379,3.946078
3,1,47,5.0,964983815,4.366379,3.975369
4,1,50,5.0,964982931,4.366379,4.237745


In [75]:
tags = pd.read_csv("D:\GITHUB\Rec_sys\Dtaset\ml-latest-small/tags.csv")
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [76]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [77]:
movie_genres = [change_string(g) for g in movies.genres.values]
movie_genres[:10]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy',
 'Action Crime Thriller',
 'Comedy Romance',
 'Adventure Children',
 'Action',
 'Action Adventure Thriller']

In [78]:
tfidf = TfidfVectorizer()
geners_tfidf = tfidf.fit_transform(movie_genres)
geners_tfidf

<9742x20 sparse matrix of type '<class 'numpy.float64'>'
	with 22084 stored elements in Compressed Sparse Row format>

In [79]:
gen_tfidf = pd.DataFrame(geners_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [80]:
movies_gen = movies.join(gen_tfidf)

In [81]:
movies_gen.head()

,movieId,title,genres,action,adventure,animation,children,comedy,crime,documentary,...,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.0,0.416846,0.516225,0.504845,0.267586,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0.0,0.512361,0.000000,0.620525,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.000000,0.000000,0.000000,0.570915,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.0,0.000000,0.000000,0.000000,0.505015,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [82]:
movies_with_tags = movies.merge(tags, on='movieId')
movies_with_tags.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,1525286013
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,1528843929
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,1528843932


In [83]:
movies_with_tags.dropna(inplace=True)

In [85]:
def change_string(s):
    return str(s).replace(' ', '').replace('-', '').replace('=', '').lower()

tag_strings = []
movies = []

for movie, group in tqdm(movies_with_tags.groupby('title')):
    tag_strings.append(' '.join([change_string(s) for s in group.tag.values]))
    movies.append(movie)

  0%|          | 0/1572 [00:00<?, ?it/s]

In [86]:
tag_strings[:10]

['artistic funny humorous inspiring intelligent quirky romance zooeydeschanel',
 'lawyers',
 'creepy suspense',
 'shakespearesortof',
 'dogs remake',
 'disney',
 'terrorism',
 'court claustrophobic confrontational earnest gooddialogue greatscreenplay gritty motivational thoughtprovoking',
 'stranded',
 'markruffalo']

In [87]:
tfidf_tag = TfidfVectorizer()
tag_tfidf = tfidf_tag.fit_transform(tag_strings)
tag_tfidf

<1572x1472 sparse matrix of type '<class 'numpy.float64'>'
	with 3598 stored elements in Compressed Sparse Row format>

In [88]:
tags_tfidf = pd.DataFrame(tag_tfidf.toarray(), columns=tfidf_tag.get_feature_names_out())
tags_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Columns: 1472 entries, 06oscarnominatedbestmovieanimation to zooeydeschanel
dtypes: float64(1472)
memory usage: 17.7 MB


In [90]:
movies_gen_tags = movies_with_tags.join(tags_tfidf)
movies_gen_tags.head()

,movieId,title,genres,userId,tag,timestamp,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.420342
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,1137206825,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,1525286013,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,1528843929,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,1528843932,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [9]:
# соединим таблицу с рейтингами и названиями фильмов
joined_ratings = ratings.join(movies.set_index('movieId'), on='movieId')